# Libraries

In [1]:
import requests
import pandas as pd
import json
from datetime import datetime as dt

# Actividad 1: Conectarse al enlace

Hago el request con la url del ejercicio

In [2]:
url_request = 'https://api.stackexchange.com/2.2/search?order=desc&sort=activity&intitle=perl&site=stackoverflow'
resp = requests.get(url_request)

Acá reviso que se haya obtenido correctamente la información

In [3]:
print(resp.status_code)

200


Guardo el contenido del request como un diccionario

In [4]:
dict_posts = json.loads(resp.text)

# Data Exploration

Los datos que nos importan se encuentran dentro de *'items'*

In [5]:
dict_posts.keys()

dict_keys(['items', 'has_more', 'quota_max', 'quota_remaining'])

Para un mejor manejo de datos transformo el diccionario a un dataframe:

In [6]:
dposts = pd.DataFrame(dict_posts['items'])

**NOTA:** Le agrego una "d" al inicio de la variable *posts* porque así sé que es un dataframe.

Veo el número total de posts:

In [7]:
total = dposts.shape[0]
print('Total de posts:', total)

Total de posts: 30


## Actividad 2: Obtener el número de respuestas contestadas y no contestadas

En esta parte no estaba seguro si utilizar la columna *'is_answered'* o *'answer_count'*, ya que al contar los True que contiene *'answer_count'* vienen 18 registros pero si cuento los *'answer_count'* diferentes de 0 son 23. 

Por lo que fui a ver el link de los post que tienen 'answer_count': False y 'answer_count' diferentes de 0 y resulta que estos en estos posts sí recibieron respuestas, por lo que decidí contar respecto a la columna *'answer_count'*.

In [8]:
answered = dposts['answer_count'].apply(lambda x: x!=0).sum()
not_answered = dposts['answer_count'].apply(lambda x: x==0).sum()

In [9]:
print(' Número de posts que recibieron respuesta:', answered, '\n Número de posts que no recibieron respuesta', not_answered)

 Número de posts que recibieron respuesta: 23 
 Número de posts que no recibieron respuesta 7


# Actividad 3: Obtener la respuesta con menor número de vistas

Ahora obtenemos el post con menor número de respuestas, acá omitiré los que no recibieron respuestas:

In [10]:
min_answers = dposts['answer_count'][dposts['answer_count']!=0].min()

In [11]:
dposts[dposts['answer_count']==min_answers].shape[0]

11

In [12]:
dposts[dposts['answer_count']==min_answers]

,tags,owner,is_answered,view_count,answer_count,score,last_activity_date,creation_date,last_edit_date,question_id,content_license,link,title,accepted_answer_id,closed_date,closed_reason,protected_date
0,"[linux, perl, ubuntu]","{'reputation': 1, 'user_id': 12918263, 'user_t...",False,29,1,-1,1656044866,1656020158,1.656020e+09,72736841,CC BY-SA 4.0,https://stackoverflow.com/questions/72736841/h...,How to Call Custom Perl Module in a Script,NaN,NaN,NaN,NaN
4,"[html, perl]","{'reputation': 103, 'user_id': 4933923, 'user_...",False,33,1,0,1655997846,1655995734,1.655998e+09,72732214,CC BY-SA 4.0,https://stackoverflow.com/questions/72732214/h...,How to fill HTML table cell colour in Perl bas...,NaN,NaN,NaN,NaN
5,"[perl, glob]","{'reputation': 1, 'user_id': 19396163, 'user_t...",False,42,1,0,1655973512,1655958468,1.655974e+09,72724534,CC BY-SA 4.0,https://stackoverflow.com/questions/72724534/u...,Using perl to delete differente file that name...,NaN,NaN,NaN,NaN
12,[perl],"{'reputation': 267, 'user_id': 1574656, 'user_...",False,10186,1,1,1655838340,1383155733,1.383161e+09,19690370,CC BY-SA 3.0,https://stackoverflow.com/questions/19690370/h...,How to redirect a page in perl script,NaN,NaN,NaN,NaN
16,"[perl, documentation]","{'reputation': 847, 'user_id': 14055985, 'user...",True,45,1,1,1655821384,1655604461,NaN,72673848,CC BY-SA 4.0,https://stackoverflow.com/questions/72673848/c...,Can Perl POD represent quoting from an email?,72702495.0,NaN,NaN,NaN
18,"[python, perl]","{'reputation': 327, 'user_id': 11397243, 'user...",False,64,1,1,1655772804,1655503268,NaN,72665191,CC BY-SA 4.0,https://stackoverflow.com/questions/72665191/m...,More perl style Function Templates in python,NaN,NaN,NaN,NaN
23,"[perl, pdl]","{'reputation': 847, 'user_id': 14055985, 'user...",True,49,1,2,1655711698,1655007107,1.655712e+09,72589437,CC BY-SA 4.0,https://stackoverflow.com/questions/72589437/d...,Does Perl PDL have an equivalent of Math::Roun...,NaN,NaN,NaN,NaN
25,[perl],"{'reputation': 18621, 'user_id': 4632019, 'use...",True,54,1,2,1655661476,1655636799,1.655650e+09,72676329,CC BY-SA 4.0,https://stackoverflow.com/questions/72676329/w...,What is corect way to configure path to packag...,72679094.0,NaN,NaN,NaN
27,[perl],"{'reputation': 3, 'user_id': 18581919, 'user_t...",True,60,1,0,1655571739,1655497868,1.655498e+09,72664514,CC BY-SA 4.0,https://stackoverflow.com/questions/72664514/p...,perl for loop issue with understanding it,72671119.0,NaN,NaN,NaN
28,"[perl, glibc, perl-packager, pp-perl-par-packa...","{'reputation': 23, 'user_id': 4783045, 'user_t...",True,42,1,1,1655561348,1655392977,1.655483e+09,72648233,CC BY-SA 4.0,https://stackoverflow.com/questions/72648233/c...,compile perl script using par::packer and run ...,NaN,NaN,NaN,NaN


# Actividad 4: Obtener la respuesta más vieja y más actual

Transformamos la columna 'creation_date' que está en formato timestamp a un formato más amigable: "AAAA-MM-DD HH:MM:SS"

In [13]:
dposts['creation_date_fix'] = dposts['creation_date'].map(dt.fromtimestamp)

Obtenemos el post más antiguo:

In [14]:
oldest_date = dposts['creation_date_fix'].min()

dposts[dposts['creation_date_fix'] == oldest_date]

,tags,owner,is_answered,view_count,answer_count,score,last_activity_date,creation_date,last_edit_date,question_id,content_license,link,title,accepted_answer_id,closed_date,closed_reason,protected_date,creation_date_fix
21,"[perl, unicode, string]","{'reputation': 1851, 'user_id': 98778, 'user_t...",True,26585,4,29,1655731309,1262022733,1.535483e+09,1970660,CC BY-SA 4.0,https://stackoverflow.com/questions/1970660/ho...,How can I guess the encoding of a string in Perl?,1974459.0,NaN,NaN,NaN,2009-12-28 11:52:13


Y el más reciente:

In [15]:
most_recent_date = dposts['creation_date_fix'].max()
dposts[dposts['creation_date_fix'] == most_recent_date]

,tags,owner,is_answered,view_count,answer_count,score,last_activity_date,creation_date,last_edit_date,question_id,content_license,link,title,accepted_answer_id,closed_date,closed_reason,protected_date,creation_date_fix
0,"[linux, perl, ubuntu]","{'reputation': 1, 'user_id': 12918263, 'user_t...",False,29,1,-1,1656044866,1656020158,1.656020e+09,72736841,CC BY-SA 4.0,https://stackoverflow.com/questions/72736841/h...,How to Call Custom Perl Module in a Script,NaN,NaN,NaN,NaN,2022-06-23 16:35:58


# Actividad 5: Obtener la respuesta del owner que tenga una mayor reputación

**NOTA**: Esta actividad no le entendí del todo, porque me pide la respuesta del owner con mayor reputación pero entiendo que solo viene el post del owner.

Para tomar la información del owner, crearé un dataframe nuevo:

In [16]:
downer = pd.DataFrame(dposts['owner'].tolist())

**NOTA:** Le agrego una "d" al inicio de la variable *owner* porque así sé que es un dataframe.

Y este es el owner con mejor reputación:

In [17]:
max_reputation = downer['reputation'].max()
downer[downer['reputation'] == max_reputation]

,reputation,user_id,user_type,profile_image,display_name,link,accept_rate
8,28046,377031,registered,https://www.gravatar.com/avatar/a9b4665d4dbfd7...,David B,https://stackoverflow.com/users/377031/david-b,92.0


De acá tomaré su nombre para obtener los posts relacionados con el owner:

In [18]:
owner_max_reputation = downer['display_name'][downer['reputation'] == max_reputation].squeeze()

In [19]:
dposts[dposts['owner'].apply(lambda x: x['display_name']==owner_max_reputation)]

,tags,owner,is_answered,view_count,answer_count,score,last_activity_date,creation_date,last_edit_date,question_id,content_license,link,title,accepted_answer_id,closed_date,closed_reason,protected_date,creation_date_fix
8,"[regex, perl]","{'reputation': 28046, 'user_id': 377031, 'user...",True,27427,7,21,1655895047,1281349700,1.493563e+09,3439123,CC BY-SA 3.0,https://stackoverflow.com/questions/3439123/ho...,How can I match everything that is after the l...,3439134.0,NaN,NaN,NaN,2010-08-09 05:28:20


# Actividad 6: Imprimir en consola del punto 2 al 5

In [20]:
print('2. Obtener el número de respuestas contestadas y no contestadas.')
print('\n\t- Número de posts que recibieron respuesta:', answered, '\n\t- Número de posts que no recibieron respuesta', not_answered)
print('\n','*'*66, '\n')

print('3. Obtener la respuesta con menor número de vistas.')
print('\n\t El menor número de respuestas obtenidas (omitiendo el cero) es:', min_answers)
print('\n\t Provenientes de los siguientes posts:')
[print('\t\t- ' + t) for t in dposts['title'][dposts['answer_count']==min_answers].values]
print('\n','*'*66, '\n')

print('4. Obtener la respuesta más vieja y más actual.\n')
[print('\t- ' + dt.strftime(r[1], '%d/%m/%Y-%H:%M') + ' - ' + r[2]) for r in dposts[['creation_date_fix', 'title']][dposts['creation_date_fix'] == oldest_date].itertuples()]
[print('\t- ' + dt.strftime(r[1], '%d/%m/%Y-%H:%M') + ' - ' + r[2]) for r in dposts[['creation_date_fix', 'title']][dposts['creation_date_fix'] == most_recent_date].itertuples()]
print('\n','*'*66, '\n')

print('5. Obtener la respuesta del owner que tenga una mayor reputación.\n')
print('\t- ' + owner_max_reputation + ': ' + dposts['title'][dposts['owner'].apply(lambda x: x['display_name']==owner_max_reputation)].squeeze())

2. Obtener el número de respuestas contestadas y no contestadas.

	- Número de posts que recibieron respuesta: 23 
	- Número de posts que no recibieron respuesta 7

 ****************************************************************** 

3. Obtener la respuesta con menor número de vistas.

	 El menor número de respuestas obtenidas (omitiendo el cero) es: 1

	 Provenientes de los siguientes posts:
		- How to Call Custom Perl Module in a Script
		- How to fill HTML table cell colour in Perl based on variable&#39;s output?
		- Using perl to delete differente file that name end by diefferent name extension Like dynamic date and hour
		- How to redirect a page in perl script
		- Can Perl POD represent quoting from an email?
		- More perl style Function Templates in python
		- Does Perl PDL have an equivalent of Math::Round::nearest()?
		- What is corect way to configure path to packages for perl?
		- perl for loop issue with understanding it
		- compile perl script using par::packer and run it